In [70]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jul 28 11:07:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   47C    P0              27W /  72W |   5147MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [71]:
%%capture
%pip install datasets>=1.18.3
%pip install transformers==4.11.3
%pip install torch
%pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
%pip install jiwer
%pip install accelerate -U
%pip install python-dotenv
%pip install librosa datasets

In [72]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) 

True

In [73]:
from huggingface_hub import notebook_login
import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder

import os
import huggingface_hub

from huggingface_hub import HfApi, HfFolder
token=os.environ.get("HF_TOKEN")
#huggingface_hub.login(token=token,add_to_git_credential=True, new_session=False, write_permission=True)

api=HfApi(token=token)
folder = HfFolder()
folder.save_token(token)



# huggingface_hub.login(token=os.environ.get("HF_TOKEN"),add_to_git_credential=True, new_session=False, write_permission=True)

In [74]:
%%capture
%apt install git-lfs

TORGO

In [75]:
# import pandas as pd
# import os
# import librosa

# def data_generator(data_dir):
#     metadata_path = os.path.join(data_dir, 'metadata.csv')
#     metadata = pd.read_csv(metadata_path)

#     for _, row in metadata.iterrows():
#         file_path = os.path.join(data_dir, row['file_name'])
#         transcription = row['transcription']
#          # Load the audio file with librosa
#         audio_data, sampling_rate = librosa.load(file_path, sr=16_000)  # Ensure the sampling rate is 16 kHz

#         # Structure the audio data and sampling rate in a dictionary
#         audio_dict = {"array": audio_data, "sampling_rate": sampling_rate}

#         yield {"audio": audio_dict, "path": file_path, "transcription": transcription}


# from datasets import Dataset, Features, Audio, Value

# data_dir = '/content/drive/MyDrive/TORGO-big'  # Adjust this path as needed

# features = Features({
#     "path": Value("string"),
#     "audio": Audio(sampling_rate=16_000),
#     "transcription": Value("string")
# })

# dataset = Dataset.from_generator(
#     generator=data_generator,
#     features=features,
#     gen_kwargs={"data_dir": data_dir}
# )


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)

# dataset = dataset.train_test_split(test_size=0.2)


UASPEECH

In [76]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [77]:
# test if pyarrow works
import pyarrow
print(pyarrow.concat_tables)

<cyfunction concat_tables at 0x7fd7718a5a40>


In [78]:
import sys
sys.executable

'/home/zeus/miniconda3/envs/cloudspace/bin/python'

In [79]:
# import pandas as pd
# import os
# import librosa
# from datasets import Dataset

# def load_parquet_directory(directory_path, use_narrow_data=False):
#     all_dfs = []
#     for file_name in os.listdir(directory_path):
#         if file_name.endswith('.parquet'):
#             file_path = os.path.join(directory_path, file_name)
#             df = pd.read_parquet(file_path)
#             print("parquet read")
#             all_dfs.append(df)
#             if use_narrow_data:
#               break

#     return pd.concat(all_dfs, ignore_index=True)

# def convert_dataframe_to_dataset(dataframe):
#     return Dataset.from_pandas(dataframe)


# # def load_audio(batch):
# #     audio_data, _ = librosa.load(batch['path'], sr=16000)
# #     batch["audio"] = audio_data
# #     return batch


# # dataset = dataset.map(load_audio)
# directory_path = '/teamspace/uploads/uaspeechall/data' # Update this path
# df = load_parquet_directory(directory_path, use_narrow_data=False)

# print("parquet loaded")
# dataset = convert_dataframe_to_dataset(df)

###### OTHER WAY


# import pyarrow.parquet as pq
# from datasets import Dataset
# import glob

# def load_memory_mapped_dataset(directory_path):
#     # Get all parquet files in the directory
#     parquet_files = glob.glob(f'{directory_path}/*.parquet')
    
#     # Read the parquet files using memory mapping
#     tables = [pq.read_table(file, memory_map=True) for file in parquet_files]
    
#     # Concatenate all tables
#     combined_table = pyarrow.concat_tables(tables)
    
#     # Convert to a Dataset object
#     return Dataset(pyarrow.Table.from_batches(combined_table.to_batches()))

# # Load the dataset
# dataset = load_memory_mapped_dataset('/teamspace/uploads/uaspeechall/data')

####### YET ANOTHER WAY
import pyarrow.parquet as pq
from datasets import Dataset, concatenate_datasets
import glob
import pyarrow as pa

def chunk_generator(table, chunk_size=10000):
    num_rows = table.num_rows
    for i in range(0, num_rows, chunk_size):
        yield table.slice(i, min(chunk_size, num_rows - i)).to_batches()[0]

def process_parquet_files(directory_path, chunk_size=10000):
    parquet_files = glob.glob(f'{directory_path}/*.parquet')
    for file in parquet_files:
        print(f"Processing file: {file}")
        table = pq.read_table(file, memory_map=True)
        for chunk in chunk_generator(table, chunk_size):
            yield Dataset(pa.Table.from_batches([chunk]))

# Usage
directory_path = '/teamspace/uploads/uaspeechall/data'
all_datasets = []

for chunk_dataset in process_parquet_files(directory_path):
    # Process each chunk here
    # For example, you can apply your transformations:
    # transformed_chunk = chunk_dataset.map(your_transform_function)
    all_datasets.append(chunk_dataset)
# Combine all chunks into a single dataset
dataset = concatenate_datasets(all_datasets)


Processing file: /teamspace/uploads/uaspeechall/data/test-00000-of-00007.parquet


Processing file: /teamspace/uploads/uaspeechall/data/test-00001-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00002-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00003-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00004-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00005-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/test-00006-of-00007.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00000-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00001-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00002-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00003-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00004-of-00018.parquet
Processing file: /teamspace/uploads/uaspeechall/data/train-00005-of-00018.parquet
Processing file: /team

In [80]:
dataset

Dataset({
    features: ['audio', 'speaker_id', 'transcription'],
    num_rows: 2500
})

In [81]:
!pip install torchvision torchaudio pydub

In [82]:
print(dataset[0]["audio"].keys())

dict_keys(['path', 'array', 'sampling_rate'])


In [83]:
# Perhaps not needed at all

import numpy as np

def convert_audio_format(batch):
    audio_data = batch["audio"]
    
    # The audio data is already in the correct format
    audio_array = np.array(audio_data["array"]).astype(np.float32)
    sampling_rate = audio_data["sampling_rate"]

    # Update the audio dictionary (this step might be unnecessary if the data is already in the correct format)
    batch["audio"] = {"array": audio_array, "sampling_rate": sampling_rate}
    return batch

# Apply the conversion function to the dataset
dataset = dataset.map(convert_audio_format)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [84]:
dataset = dataset.train_test_split(test_size=0.2)

In [85]:
from datasets import load_dataset, load_metric, Audio

train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [86]:
train_dataset[0]

{'audio': {'path': None,
  'array': array([-6.10351562e-05, -2.44140625e-04, -2.13623047e-04, ...,
         -1.52587891e-04,  0.00000000e+00,  0.00000000e+00]),
  'sampling_rate': 16000},
 'speaker_id': 'M14_B1_UW91_M2',
 'transcription': 'wagon'}

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [87]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [88]:
# import soundfile as sf
# import numpy as np

# def save_audio_samples(dataset, num_samples=5):
#     for i in range(num_samples):
#         index = np.random.randint(0, len(dataset))
#         example = dataset[index]
#         audio_data = example["audio"]
        
#         if isinstance(audio_data, dict) and "array" in audio_data and "sampling_rate" in audio_data:
#             audio_array = audio_data["array"]
#             sampling_rate = audio_data["sampling_rate"]
            
#             filename = f"audio_sample_{i}.wav"
#             sf.write(filename, audio_array, sampling_rate)
#             print(f"Saved {filename}")
#             print(f"Transcription: {example['transcription']}")
#             print(f"Speaker ID: {example['speaker_id']}")
#             print("="*50)

# # Use the function
# save_audio_samples(train_dataset, num_samples=5)

In [89]:
import numpy as np
import soundfile as sf
from IPython.display import display, Audio, HTML
import os

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    for i in range(num_examples):
        index = np.random.randint(0, len(dataset))
        example = dataset[index]
        
        print(f"Example {i+1}:")
        print(f"Transcription: {example['transcription']}")
        print(f"Speaker ID: {example['speaker_id']}")
        
        audio_data = example["audio"]
        if isinstance(audio_data, dict) and "array" in audio_data and "sampling_rate" in audio_data:
            audio_array = audio_data["array"]
            sampling_rate = audio_data["sampling_rate"]
            
            print(f"Audio length: {len(audio_array) / sampling_rate:.2f} seconds")
            
            # Save as WAV file
            filename = f"audio_sample_{i}.wav"
            sf.write(filename, audio_array, sampling_rate)
            print(f"Audio saved as: {filename}")
            
            # Display audio player (this may or may not work depending on your environment)
            display(Audio(data=audio_array, rate=sampling_rate))
            
            # Provide a download link
            file_path = os.path.abspath(filename)
            display(HTML(f'<a href="file://{file_path}" download="{filename}">Download {filename}</a>'))
        
        else:
            print("Audio: Unexpected format")
        
        print("\n" + "="*50 + "\n")

# Use the function
show_random_elements(train_dataset, num_examples=1)

Example 1:
Transcription: all
Speaker ID: F05_B2_CW33_M2
Audio length: 2.05 seconds
Audio saved as: audio_sample_0.wav


In [90]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

In [91]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [92]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [93]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [94]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [95]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [96]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [97]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [98]:
vocab_dict

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '|': 0,
 '[UNK]': 27,
 '[PAD]': 28}

In [99]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [100]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [101]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(vocab_file="vocab.json", vocab_dict=vocab_dict, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [102]:
print("Tokenizer vocabulary size:", len(tokenizer))
print("Special tokens:", tokenizer.special_tokens_map)
print("All tokens:", list(tokenizer.get_vocab().keys()))



Tokenizer vocabulary size: 31
Special tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}
All tokens: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '[UNK]', '[PAD]', '<s>', '</s>']


In [103]:
repo_name = "wav2vec2-large-960h-lv60-self-dysarthria"
user_name = "jmaczan"

In [104]:
tokenizer.push_to_hub(repo_name, token=token)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-960h-lv60-self-dysarthria/commit/546425ddca61f120a08a0aa09959a56d0d38223e', commit_message='Upload tokenizer', commit_description='', oid='546425ddca61f120a08a0aa09959a56d0d38223e', pr_url=None, pr_revision=None, pr_num=None)

In [105]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [106]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [107]:
train_dataset[0]["audio"]

{'path': None,
 'array': array([-6.10351562e-05, -2.44140625e-04, -2.13623047e-04, ...,
        -1.52587891e-04,  0.00000000e+00,  0.00000000e+00]),
 'sampling_rate': 16000}

In [108]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset)-1)

print(train_dataset[rand_int]["transcription"])
ipd.Audio(data=train_dataset[rand_int]["audio"]["array"], autoplay=False, rate=16000)

authoritative


In [109]:
rand_int = random.randint(0, len(train_dataset)-1)

print("Target text:", train_dataset[rand_int]["transcription"])
print("Input array shape:", train_dataset[rand_int]["audio"]["array"])#.shape)
print("Sampling rate:", train_dataset[rand_int]["audio"]["sampling_rate"])

Target text: make
Input array shape: [ 1.83105469e-04 -1.22070312e-04 -4.88281250e-04 ...  3.05175781e-05
 -6.10351562e-05  0.00000000e+00]
Sampling rate: 16000


In [110]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [112]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Training


In [113]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [114]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [115]:
wer_metric = load_metric("wer")

In [116]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [117]:
!pip install -U flash-attn --no-build-isolation

In [118]:
vocab_dict

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '|': 0,
 '[UNK]': 27,
 '[PAD]': 28}

In [119]:


processor.tokenizer.add_special_tokens({'pad_token': '[PAD]', 'unk_token': '[UNK]'})

# Remove 'bos_token' and 'eos_token' from special_tokens_map
del processor.tokenizer.special_tokens_map['bos_token']
del processor.tokenizer.special_tokens_map['eos_token']

processor.tokenizer


Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=29, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	27: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	28: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	29: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [120]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h-lv60-self",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.1,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    ignore_mismatched_sizes=True
).to('cuda')

Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.maske

In [121]:
model.freeze_feature_extractor()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [122]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=1,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=False,
  bf16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=100,
  learning_rate=1e-4,
  warmup_steps=1000,
  save_total_limit=5,
  push_to_hub=True,
  weight_decay=0.01,
  lr_scheduler_type="cosine",
  load_best_model_at_end=True,
  metric_for_best_model="wer"
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [123]:
from transformers import Trainer, EarlyStoppingCallback, AdamW

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    optimizers=(optimizer, None)
    # callbacks=[early_stopping_callback]
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Training

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [124]:
trainer.train()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
200,102.559500,96.484894,1.044000
400,96.315200,90.386360,1.030000
600,86.931200,76.048439,1.004000
800,62.594200,48.233360,1.000000
1000,38.327800,23.973984,1.000000
1200,5.511700,3.463387,1.000000
1400,4.946500,3.377197,1.000000
1600,3.584100,3.359337,1.000000
1800,3.973100,3.345942,1.000000
2000,3.488600,3.337078,1.000000


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: U

TrainOutput(global_step=3750, training_loss=24.735951456705727, metrics={'train_runtime': 2182.2241, 'train_samples_per_second': 27.495, 'train_steps_per_second': 1.718, 'total_flos': 5.941437836096738e+18, 'train_loss': 24.735951456705727, 'epoch': 30.0})

In [125]:
trainer.push_to_hub()

events.out.tfevents.1722165207.ip-10-192-10-202.52583.3:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jmaczan/wav2vec2-large-960h-lv60-self-dysarthria/commit/769e66bd504661ff424cc42e6b0de12c57239bad', commit_message='End of training', commit_description='', oid='769e66bd504661ff424cc42e6b0de12c57239bad', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-large-xls-r-300m-tr-colab")
```

### Evaluation

In [126]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

In [127]:
input_dict = processor(test_dataset[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [128]:
df = load_parquet_directory(directory_path, use_narrow_data=True)
dataset = convert_dataframe_to_dataset(df)

test_dataset_transcription = dataset.train_test_split(test_size=0.2)["test"]

NameError: name 'load_parquet_directory' is not defined

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_dataset_transcription[0]["transcription"].lower())

### Optional 
Further training phase on next parquet file

In [ ]:
import pandas as pd
import os
import librosa
from datasets import Dataset

global_parquet_file_index = 1

def load_parquet_directory_by_file_index(directory_path, parquet_file_index=1):
    all_dfs = []
    current_parquet_file_index = 0
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.parquet'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_parquet(file_path)
            all_dfs.append(df)
            if current_parquet_file_index == parquet_file_index:
              break
            current_parquet_file_index = current_parquet_file_index + 1

    return pd.concat(all_dfs, ignore_index=True)

def convert_dataframe_to_dataset(dataframe):
    return Dataset.from_pandas(dataframe)


# def load_audio(batch):
#     audio_data, _ = librosa.load(batch['path'], sr=16000)
#     batch["audio"] = audio_data
#     return batch


# dataset = dataset.map(load_audio)
directory_path = '/content/drive/MyDrive/uaspeechall/data'
df = load_parquet_directory_by_file_index(directory_path, parquet_file_index=True)
dataset = convert_dataframe_to_dataset(df)




In [ ]:
import io
import numpy as np
from pydub import AudioSegment

def convert_audio_format(batch):
    # Extract the audio data from the dictionary
    audio_bytes = batch["audio"]["bytes"]  # Replace "data" with the correct key

    # Convert binary data to audio array
    audio_data = AudioSegment.from_file(io.BytesIO(audio_bytes), format="wav")
    # Convert to a NumPy array and normalize
    audio_array = np.array(audio_data.get_array_of_samples()).astype(np.float32) / 2**15

    # Update the audio dictionary
    batch["audio"] = {"array": audio_array, "sampling_rate": 16000}
    return batch

# Apply the conversion function to the dataset
dataset = dataset.map(convert_audio_format)

from datasets import load_dataset, load_metric, Audio

dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

tokenizer.push_to_hub(repo_name)

from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

model = Wav2Vec2ForCTC.from_pretrained(
    user_name+"/"+repo_name,
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()
trainer.push_to_hub()
